In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [9]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.85024634e-03  1.65567698e-03 -1.81738939e-03 ... -2.44658417e-03
   -2.74311984e-03 -4.65289317e-03]
  [-3.01057613e-03  3.14018317e-03 -1.48372899e-04 ... -1.78218016e-03
   -1.03205140e-03 -5.00329956e-03]
  [ 6.39770122e-04 -2.39068642e-04 -9.00931284e-03 ...  1.70725095e-03
   -1.16185925e-03  1.68139185e-03]
  ...
  [ 1.09125802e-03  5.71241090e-03 -4.59420355e-03 ... -8.81332438e-04
   -1.40497107e-02  1.13486983e-02]
  [ 3.67151666e-03  4.18187073e-03  3.95439006e-03 ...  7.50035699e-03
   -2.94375629e-03  1.04372045e-02]
  [ 8.05591932e-04  4.68742289e-03  2.69060070e-03 ...  6.62863674e-03
   -5.25720092e-03  6.07130025e-03]]

 [[-6.07479876e-03  6.85411552e-03 -1.74195040e-04 ... -2.23765220e-03
   -3.45840096e-03 -4.12077177e-03]
  [-6.18601963e-03  6.62945397e-03  2.09354120e-03 ... -1.95917767e-03
   -1.77946640e-03 -4.56039049e-03]
  [-6.66042930e-03  4.90491930e-03  1.71975873e-03 ...  1.55093381e-03
   -3.80370766e-03 -3.11958371e-03]
  ...
  [-3.409

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00185025  0.00165568 -0.00181739 ... -0.00244658 -0.00274312
  -0.00465289]
 [-0.00301058  0.00314018 -0.00014837 ... -0.00178218 -0.00103205
  -0.0050033 ]
 [ 0.00063977 -0.00023907 -0.00900931 ...  0.00170725 -0.00116186
   0.00168139]
 ...
 [ 0.00109126  0.00571241 -0.0045942  ... -0.00088133 -0.01404971
   0.0113487 ]
 [ 0.00367152  0.00418187  0.00395439 ...  0.00750036 -0.00294376
   0.0104372 ]
 [ 0.00080559  0.00468742  0.0026906  ...  0.00662864 -0.0052572
   0.0060713 ]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.8502463e-03  1.6556770e-03 -1.8173894e-03  3.3495473e-03
 -2.1389248e-03  5.9776660e-03 -1.1803122e-03 -5.5387453e-03
 -1.4418662e-03 -8.3111897e-03  3.1458493e-03 -2.4265854e-03
 -6.0621067e-03 -4.6094409e-03 -8.5900980e-04  7.4464013e-04
  8.9333393e-04 -3.0684465e-04 -2.7248282e-03 -8.7916228e-04
  4.7804439e-03 -3.3156497e-03 -3.7552756e-03  5.5657438e-04
  1.5526661e-04 -6.5634027e-04  1.5822034e-03 -2.3365470e-03
  3.6437116e-03  1.4208192e-03  3.2233533e-03 -3.5853623e-03
  2.3739694e-03  1.3485546e-03 -2.7355370e-03 -7.8930613e-03
  2.2439999e-03  3.3515105e-03 -3.5895230e-03 -1.8390790e-03
 -6.2095453e-03 -4.1148491e-04 -3.7809990e-03  1.9820258e-03
  5.1316514e-05  3.1317929e-03 -9.5966383e-04 -2.6842644e-03
  3.2864604e-03 -1.4403842e-03 -9.7981468e-04 -7.7154534e-04
 -4.0753963e-03  3.0712129e-03  6.5891230e-03  1.9171652e-03
 -4.4065779e-03 -1.0739077e-03 -1.4960145e-03 -1.7669357e-03
 -5.0405990e-03  3.4194693e-03 -2.4465842e-03 -2.7431198e-03
 -4.652893

In [21]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'IE&dmMbxbEfQ-Bo:WYbtTQ.UIrlCrT;C.XVzkgGfJGSvIcmmsmvikfd bEalx,KRRnG aQv$ RbGIOYcVs,sC3?YS,hB?xRIxnl&'

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [23]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
history = model.fit(data, epochs=1, callbacks=[checkpoint_callback]) #Use a higher number of epochs for better results.

172/172 [==============================] - 1339s 8s/step - loss: 1.7308


In [28]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [29]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [33]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [36]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: I love natural language processing
I love natural language processings.

LIONANEL:
ULOuld with my:
And, what is the knors: waccest the ruent home
concullent me have comemity.

Suchard: ichinare, bode! so nigh himself, what he
jows, nothing be and on frow san was frecturs, theref,
Os with meetion and call a Vorsence?
Call have say and thot spidition,
Too Inten amanies, steak nif
Unour dos
Hay spike, but fie it pay nol I 'ever her war,
I'll oper the tearn'd bear my Greath.

Coming Mormar:
I lode thos nobthible go?

GREUCE:
Ame in morery.

Lere and seem the doart:
'tis forse that had stinkent he prieven:
Ancenty mine edy aldy morrisch; but thoreating
Tarking me knoparous with at thy manier:
Rore hadse I read, but leriesp,
Come, to but in spest the greechers steat.

Thar IVIND:
Seeve love! who his drainst'd nothing will her enverges
To his in hat honour brother
